In [1]:
# libraries and settings
import os
import c3d
import glob2
import math
import numpy as np
from mayavi.mlab import *
import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.animation as animation
from matplotlib import animation, rc
from mpl_toolkits.mplot3d import Axes3D

from IPython.display import HTML, Image

%matplotlib qt

rc('animation', html='html5')

In [2]:
# try to read c3d files
folder_name = 'files_motions_884'
cd3data_path = os.path.join('data', 'motion_data', folder_name)
all_files = glob2.glob(cd3data_path+'/**/*[0-9].c3d', recursive=True)
for files in all_files: 
    print(files)

# read c3d file
reader = c3d.Reader(open(all_files[0], 'rb'))

data/motion_data/files_motions_884/pour_and_mixing_normal_speed_03.c3d
data/motion_data/files_motions_884/pour_and_mixing_normal_speed_01.c3d
data/motion_data/files_motions_884/pour_and_mixing_normal_speed_02.c3d


### See information about marker points [here](https://motion-database.humanoids.kit.edu/marker_set/)

In [3]:
# reader.point_used
point_labels = reader.point_labels
# print(type(point_labels))
print(reader.point_labels)

[u'sponge_small:sponge_dry_01    ', u'sponge_small:sponge_dry_02    ', u'sponge_small:sponge_dry_03    ', u'sponge_small:sponge_dry_04    ', u'sponge_big:sponge_wet_01      ', u'sponge_big:sponge_wet_02      ', u'sponge_big:sponge_wet_03      ', u'sponge_big:sponge_wet_04      ', u'small_cup:sc_01               ', u'small_cup:sc_02               ', u'small_cup:sc_03               ', u'small_cup:sc_04               ', u'mixing_bowl_small:mbs_01      ', u'mixing_bowl_small:mbs_02      ', u'mixing_bowl_small:mbs_03      ', u'mixing_bowl_small:mbs_04      ', u'mixing_bowl_big:mb_01         ', u'mixing_bowl_big:mb_02         ', u'mixing_bowl_big:mb_03         ', u'mixing_bowl_big:mb_04         ', u'knife:knife_01                ', u'knife:knife_02                ', u'knife:knife_03                ', u'knife:knife_04                ', u'egg_whisk:ew_01               ', u'egg_whisk:ew_02               ', u'egg_whisk:ew_03               ', u'egg_whisk:ew_04               ', u'cooking_spoon:sho

In [4]:
# print(type(point_labels))
# print(point_labels[:40])
print(point_labels[49:56])
print(point_labels[58:65])
print(point_labels[62:64])

[u'425:LSHO                      ', u'425:LUPA                      ', u'425:LAEL                      ', u'425:LAOL                      ', u'425:LWTS                      ', u'425:LWPS                      ', u'425:LFRA                      ']
[u'425:RSHO                      ', u'425:RUPA                      ', u'425:RAEL                      ', u'425:RAOL                      ', u'425:RWTS                      ', u'425:RWPS                      ', u'425:RFRA                      ']
[u'425:RWTS                      ', u'425:RWPS                      ']


In [5]:
objs_id_end = 40
lh_id_start = 49
rh_id_start = 58

In [6]:
# the marker points that we need
# left_arm_markers = ['LSHO', 'LUPA', 'LAEL', 'LAOL', 'LWTS', 'LWPS', 'LIFD']
# right_arm_markers = ['RSHO', 'RUPA', 'RAEL', 'RAOL', 'RWTS', 'RWPS', 'RIFD']
# print(point_labels[:12])
object_points_ids = list(range(objs_id_end))
left_arm_points_ids = list(range(lh_id_start,lh_id_start+7)) #28
right_arm_points_ids = list(range(rh_id_start,rh_id_start+7)) #38

lh_traj_points_id = list(range(lh_id_start+4,lh_id_start+4+2))
rh_traj_points_id = list(range(rh_id_start+4,rh_id_start+4+2))

scene_groups_ids = [object_points_ids, left_arm_points_ids, right_arm_points_ids]
traj_groups_ids = [lh_traj_points_id, rh_traj_points_id]

In [7]:
# the following part seems expensive, but we have to read the frames one at a time
scene_groups = []
for group in scene_groups_ids:
    group_points = []
    for point_id in group:
        point_marker = []
        for i, points, analog in reader.read_frames():
            point_marker.append(points[point_id][:3])
        group_points.append(point_marker)
    scene_groups.append(np.array(group_points))

In [8]:
# for trajectory points that we want to trace
traj_groups = []
for group in traj_groups_ids:
    traj_points = []
    for point_id in group:
        point_marker = []
        for i, points, analog in reader.read_frames():
            traj_mrkrs = points[point_id][:3]
            point_marker.append(traj_mrkrs)
        traj_points.append(point_marker)
    traj_groups.append(np.array(traj_points))

In [9]:
# we want to create trace coordinates from the traj_points
for group in traj_groups:
    point_a = group[0]
    point_b = group[1]
    mid_point = (point_a + point_b)/2.0
    
print(point_a[0])
print(point_b[0])
print(mid_point[0])

[ 839.4598999   400.67034912  861.78747559]
[ 789.38293457  370.97470093  859.44543457]
[ 814.42141724  385.82252502  860.61645508]


In [10]:
total_steps = len(mid_point)

In [11]:
len(scene_groups[2])

7

### visualize a frame

### we are going to try to make animations of the arm movement

In [20]:
grp_nos = 3
def update_lines(num, dataLines, lines):
    for i,groups in enumerate(scene_groups[:grp_nos]):
        frame = groups[:,num,:]

        x_data = frame[:,0]
        y_data = frame[:,1]
        z_data = frame[:,2]
        graph_list[i]._offsets3d = (x_data, y_data, z_data)
        
    title.set_text('Wrist Path Trace, time step={}'.format(num))
        
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
# ax = p3.Axes3D(fig)
ax = fig.add_subplot(111, projection='3d')


# the data
data = [np.transpose(mid_point)]

# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1], linewidth=2)[0] for dat in data]

# plot initial data for scatter points
colours = ['r', 'b', 'g', 'k']
graph_list = []
for i, groups in enumerate(scene_groups[:grp_nos]):
    x_data = groups[:,0,:][:,0]
    y_data = groups[:,0,:][:,1]
    z_data = groups[:,0,:][:,2]
    graph = ax.scatter(x_data, y_data, z_data, c=colours[i])
    graph_list.append(graph)

title = ax.set_title('Wrist Path Trace')
# Setting the axes properties
ax.set_xlim3d([-1700.0, 0.0])
ax.set_xlabel('X')

ax.set_ylim3d([-600.0, 800.0])
ax.set_ylabel('Y')

ax.set_zlim3d([700.0, 1600.0])
ax.set_zlabel('Z')
ax.azim = 180
ax.elev = 30

# Creating the Animation object
ani = animation.FuncAnimation(fig, update_lines, total_steps, fargs=(data, lines),
                                   interval=0, blit=False)

plt.show()

# ani.save('data/media_outputs/'+folder_name+'_trace.mp4', writer=animation.FFMpegWriter(fps=30, bitrate=1000), fps=30)
# ani.save('data/media_outputs/'+folder_name+'_trace.gif', writer='imagemagick', fps=30)

### Visualize demo points only

In [ ]:
# display image or animation through here:
# Image(url='data/media_outputs/basic_animation.gif')